In [1]:
import numpy as np

# Import local plotting functions and in-notebook display functions
import matplotlib.pyplot as plt
%matplotlib inline


# the Lorenz Model

Let's use a famous set of coupled differential equations to demonstrate our points

## Explicit fixed-step integrators

In [ ]:

class BaseFixedStepIntegrator:
    """
    A base class for fixed-step integration methods.
    """
    def __init__(self, dt=1e-3):
        self.dt = dt
        self.name = self.__class__.__name__
        

class Euler(BaseFixedStepIntegrator):
    """
    Note:
    super() calls the constructor of BaseIntegrator
    kwargs get passed on to the constructor of BaseIntegrator
    """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def integrate(self, f, t0, y0, t1):
        """
        Integrate the ODE y' = f(t, y) from t0 to t1 using Euler's method.
        """
        t = np.arange(t0, t1, self.dt)
        n = len(t)
        y = np.zeros((n + 1, y0.shape[-1]))
        y[0] = y0
        print(y[0])
        for i in range(n):
            y[i + 1] = y[i] + self.dt * f(t[i], y[i])
        return t, y

class RungeKutta(BaseFixedStepIntegrator):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def integrate(self, f, t0, y0, t1):
        t = np.arange(t0, t1, self.dt)
        n = len(t)
        y = np.zeros((n + 1, y0.shape[-1]))
        y[0] = y0
        for i in range(n):
            k1 = f(t[i], y[i])
            k2 = f(t[i] + 0.5 *  self.dt, y[i] + 0.5 *  self.dt * k1)
            k3 = f(t[i] + 0.5 *  self.dt, y[i] + 0.5 *  self.dt * k2)
            k4 = f(t[i] + self.dt, y[i] + self.dt * k3)
            y[i + 1] = y[i] + self.dt / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
        return t, y


class DormandPrince(BaseFixedStepIntegrator):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def integrate(self, f, t0, y0, t1):
        t = np.arange(t0, t1, self.dt)
        h = self.dt
        n = len(t)
        y = np.zeros((n + 1, y0.shape[-1]))
        y[0] = y0
        for i in range(n):
            k1 = f(t[i], y[i])
            k2 = f(t[i] + 0.25 * h, y[i] + 0.25 * h * k1)
            k3 = f(t[i] + (3 / 8) * h, y[i] + (3 / 32) * h * k1 + (9 / 32) * h * k2)
            k4 = f(t[i] + (12 / 13) * h, y[i] + (1932 / 2197) * h * k1 - (7200 / 2197) * h * k2 + (7296 / 2197) * h * k3)
            k5 = f(t[i] + h, y[i] + (439 / 216) * h * k1 - 8 * h * k2 + (3680 / 513) * h * k3 - (845 / 4104) * h * k4)
            k6 = f(t[i] + (1 / 35) * h, y[i] - (8 / 27) * h * k1 + 2 * h * k2 - 3544 / 2565 * h * k3 + 1859 / 4104 * h * k4 - 11 / 40 * h * k5)
            y[i + 1] = y[i] + (25 / 216) * h * k1 + 1408 / 2565 * h * k3 + 2197 / 4104 * h * k4 - 1 / 5 * h * k5
        return t, y


## Butcher Tableau

A recursive set of definitions for the coefficients in explicit Runge-Kutta methods

<!-- \begin{array} {c|cccc} 0\\ 1/2 & 1/2\\ 1/2 &0 &1/2 \\ 1& 0& 0& 1\\ \hline & 1/6 & 1/3 & 1/3 &1/6 \end{array}

\begin{array} {c|cccc} 0\\ 1/3 & 1/3\\ 2/3 & -1/3 &1 \\ 1& 1& -1 & 1\\ \hline & 1/8 & 3/8 & 3/8 &1/8 \end{array}

\begin{array} {c|ccccc} 0\\ c_2 & a_{21}\\ c_3 & a_{31} & a_{32} \\ \vdots & \vdots & & \ddots \\ c_s& a_{s1}& a_{s2} & \cdots & a_{s,s-1}\\ \hline & b_1 & b_2 & \cdots & b_{s-1} & b_s \end{array}

 -->

$$
y' = f(t, y)
$$

$$
y_{n+1} = y_n + \frac{h}{6}\left( k_{n1} + 2k_{n2} + 2k_{n3} + k_{n4}\right)
$$


k_{n1} &=& f(t_n, y_n) \\ 
k_{n2} &=& f(t_n + 0.5h, y_n + 0.5hk_{n1}) \\ 
k_{n3} &=& f(t_n + 0.5h, y_n + 0.5hk_{n2}) \\
k_{n4} &=& f(t_n + h, y_n + hk_{n3}) \\


<!-- k_{n1} &=& f(t_n, y_n) \\ k_{n2} &=& f(t_n + 0.5h, y_n + 0.5hk_{n1}) \\ k_{n3} &=& f(t_n + 0.5h, y_n + 0.5hk_{n2}) \\ k_{n4} &=& f(t_n + h, y_n + hk_{n3}) \\ -->

## Runge-Kutta methods

$$
k_{ni} = f\left(x_n + c_i h, y_n + h \sum_{j=1}^s a_{ij} k_{nj}\right)
$$

#### Euler's method (forward step)

\begin{array} 
{c|c} 0 & 0\\ \hline & 1
\end{array}

#### Euler's method (backward step)

\begin{array} 
{c|c} 1 & 1\\ \hline & 1
\end{array}





\begin{array} {c|ccccc} c_1 & a_{11} & a_{12} & \cdots & a_{1s} \\ c_2 & a_{21} & a_{22} & \cdots & a_{2s} \\ \vdots & \vdots & & \ddots & \vdots \\ c_s& a_{s1}& a_{s2} & \cdots & a_{s,s}\\ \hline & b_1 & b_2 & \cdots & b_{s} \end{array}


Notes here:
https://www.johndcook.com/blog/2020/02/13/runge-kutta-methods/

# The Butcher Tableau, Implicit

# Linear multistep

# Variable step

# Symplectic

When we have an explict conserved quantity, many of the above methods will fail

